In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from shutil import copyfile
from PIL import Image
# import os
import tensorflow as tf
%matplotlib inline
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
import os
import skimage.io as io
from ipywidgets import interact, Text, RadioButtons
import ipywidgets as widgets
from IPython.display import clear_output
# %matplotlib inline

In [ ]:
def read_from_tf_records(filenames, mode=None):
    """
    Reads images, labels from tfrecords files.
    :param mode: set to 'pred', if there are predicted labels in tfrecords file
    :param filenames: list of tfrecord file names
    :return: lists of
    """
    img_heights, img_widths, dice_scores, groves, imgs, masks, results, img_names = [], [], [], [], [], [], [], []
#     if type(filenames) == str:
#         filenames = list(filenames)

    # for filename in filenames:
        # print(filename)
    assert os.path.isfile(filenames)
    for serialized_example in tf.python_io.tf_record_iterator(filenames):
        example = tf.train.Example()
        example.ParseFromString(serialized_example)
        # ---
        img_raw = example.features.feature['img_raw'].bytes_list.value
        msk_raw = example.features.feature['mask_raw'].bytes_list.value
        rst_raw = example.features.feature['result_raw'].bytes_list.value
        img_height = example.features.feature['img_height'].int64_list.value
        img_width = example.features.feature['img_width'].int64_list.value
        image_name = example.features.feature['img_name'].bytes_list.value
        grove = example.features.feature['grove'].bytes_list.value
        dice_score = example.features.feature['dice_score'].float_list.value
        # ----
        # print(img_height, img_width)
        # print(img_raw)
        image = Image.frombytes('RGB', (img_width[0], img_height[0]), img_raw[0])
        imgs.append(np.array(image))
        # print(msk_raw)
        mask = Image.frombytes('L', (img_width[0], img_height[0]), msk_raw[0])
        masks.append(np.array(mask))
        result = Image.frombytes('L', (img_width[0], img_height[0]), rst_raw[0])
        results.append(np.array(result))
        # ----
        img_heights.append(img_height[0])
        img_widths.append(img_width[0])
        dice_scores.append(dice_score[0])
        groves.append(grove[0])
        img_names.append(image_name[0])
#             if mode == 'pred':
#                 labels_pred.append(example.features.feature['label_pred'].int64_list.value)
#                 confidence.append(example.features.feature['confidence'].float_list.value)

    return img_heights, img_widths, np.array(dice_scores), np.array(groves), np.array(imgs), np.array(masks), np.array(results), np.array(img_names)

In [ ]:

def visualize_tfr(image, mask, result, dices, names):
    grid_width=2
    grid_height=6

    fig = plt.figure(figsize=(20, 22))
    outer = gridspec.GridSpec(grid_height, grid_width, wspace=0.1, hspace=0.1)

    for i in range(grid_height * grid_width):
        inner = gridspec.GridSpecFromSubplotSpec(1, 3,
                        subplot_spec=outer[i], wspace=0.1, hspace=0.1)
#         if len(image[i]) < grid_height * grid_width:
#             np.zeros((grid_height * grid_width))
        try:
            for j, (im, cls, label) in enumerate(zip([image[i], mask[i], result[i]],
                                                     ['Image', 'Mask', 'Result'],
                                                     [dices[i], names[i], names[i]])):
                ax = plt.Subplot(fig, inner[j])
                ax.set_title(cls)
                if cls == 'Image':
                    ax.imshow(im)

                elif cls == 'Mask':
                    ax.imshow(im, cmap=plt.cm.gray, interpolation='nearest')
                    ax.set_xlabel(str(names[i]) + '_' + str(round(dices[i], 4)))
                else:
                    ax.imshow(im, cmap=plt.cm.gray, interpolation='nearest')

                ax.set_xticks([])
                ax.set_yticks([])
                fig.add_subplot(ax)
        except:
            continue

    # fig.show()

In [ ]:
def visualize_tf_records_inter(tfr, min_thresh=0, max_thresh=1, num_samples=18):
    img_heights, img_widths, dice_scores, groves, imgs, masks, results, img_names = read_from_tf_records(tfr)
    mean_dice_by_groves = []
    for gr in set(groves):
        mean_dice_by_groves.append(np.mean(dice_scores[np.where(groves == gr)[0]]))
        # print(mean_dice_by_groves)
    # opt = set(b'all').update(set(pgroves))
    opt = set(groves)
    opt.add(b'all')
    # rint(opt)
    rbtn = RadioButtons(options=opt)
    display(rbtn)
    def on_value_change(change):
        clear_output(True)
        print('mean_dice')
        for gr, dc in zip(set(groves), mean_dice_by_groves):
            print(gr, ':', dc)
        display(rbtn)
        step = num_samples
        if change['new'] == b'all':
            indxs = np.where((dice_scores > min_thresh) & (dice_scores < max_thresh))[0]
            parts = int(np.ceil(len(indxs)/step))
        else:
            indxs = np.where((dice_scores > min_thresh) & (dice_scores < max_thresh) & ((groves == change['new'])))[0]
            parts = int(np.ceil(len(indxs)/step))
        # print(parts)
        n = len(indxs)
        print('Number of images: {}'.format(n))
        if n < 1:
            print('Nothing to display')
            return None
        def view_image(i):
            start = i*step
            finish = step + i*step
            ind = indxs[start:finish]
            # grid_height = 6# int(np.ceil(len(ind)/3))
            visualize_tfr(imgs[ind], masks[ind], results[ind], dice_scores[ind], img_names[ind])
            
        interact(view_image, i=(0,parts-1), value=0)
    rbtn.observe(on_value_change, names='value')

In [ ]:
visualize_tf_records_inter('/home/user/projects/geo/polygonization-nn/data/mixed_nrg_127.tfrecords',
                           min_thresh=0.0,
                           max_thresh=0.5,
                           num_samples = 8)

In [ ]:
visualize_tf_records_inter('/mnt/storage_4tb/ymi/geo_data/test_instance_plus/semantic/two_heads.tfrecords',
                           min_thresh=0.0,
                           max_thresh=1,
                           num_samples = 8)

In [ ]:
visualize_tf_records_inter('/mnt/storage_4tb/ymi/geo_data/test_instance_plus/semantic/one_head.tfrecords',
                           min_thresh=0.0,
                           max_thresh=1,
                           num_samples = 8)

In [ ]:
visualize_tf_records_inter('/home/user/projects/geo/dl/unet/data/test_weights/df_tr_nrg_train_weighted_27_09.tfrecords',
                           min_thresh=0.7,
                           max_thresh=0.8,
                           num_samples = 8)